<a href="https://colab.research.google.com/github/ananyarao23/EmoSense-TweetSentimentDetection/blob/main/week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/drive/MyDrive/WiDS-2023/week1_df1.csv')


In [4]:
df = df.dropna()
df

,Unnamed: 0,Sentiment,SentimentText,SentimentText_stopword,TokenizedText,StemmedText,LemmatizedText,Text_Subjectivity,Text_Polarity,Text_Analysis
0,0,0,is so sad for my apl friend,sad apl friend,"['sad', 'apl', 'friend']",sad apl friend,sad apl friend,1.000000,-0.500000,Negative
1,1,0,i missed the new moon trailer,missed new moon trailer,"['missed', 'new', 'moon', 'trailer']",miss new moon trailer,missed new moon trailer,0.454545,0.136364,Positive
2,2,1,omg its already 7 30 o,omg already 7 30,"['omg', 'already', '7', '30']",omg alreadi 7 30,omg already 7 30,0.000000,0.000000,Neutral
3,3,0,omgaga im sooo im gunna cry i ve been at th...,omgaga im sooo im gunna cry dentist since 11 s...,"['omgaga', 'im', 'sooo', 'im', 'gunna', 'cry',...",omgaga im sooo im gunna cri dentist sinc 11 su...,omgaga im sooo im gunna cry dentist since 11 s...,0.000000,0.000000,Neutral
4,4,0,i think mi bf is cheating on me t t,think mi bf cheating,"['think', 'mi', 'bf', 'cheating']",think mi bf cheat,think mi bf cheating,0.000000,0.000000,Neutral
...,...,...,...,...,...,...,...,...,...,...
1578607,1578607,1,zzzzzz finally night tweeters,zzzzzz finally night tweeters,"['zzzzzz', 'finally', 'night', 'tweeters']",zzzzzz final night tweeter,zzzzzz finally night tweeter,1.000000,0.000000,Neutral
1578608,1578608,1,zzzzzzz sleep well people,zzzzzzz sleep well people,"['zzzzzzz', 'sleep', 'well', 'people']",zzzzzzz sleep well peopl,zzzzzzz sleep well people,0.000000,0.000000,Neutral
1578609,1578609,0,zzzzzzzzzz wait no i have homework,zzzzzzzzzz wait homework,"['zzzzzzzzzz', 'wait', 'homework']",zzzzzzzzzz wait homework,zzzzzzzzzz wait homework,0.000000,0.000000,Neutral
1578610,1578610,0,zzzzzzzzzzzzz meh what am i doing up again,zzzzzzzzzzzzz meh,"['zzzzzzzzzzzzz', 'meh']",zzzzzzzzzzzzz meh,zzzzzzzzzzzzz meh,0.000000,0.000000,Neutral


In [23]:
from sklearn.model_selection import train_test_split

# train_text, temp_text, train_labels, temp_labels = train_test_split(df['LemmatizedText'],df['Sentiment'],random_state=2024,test_size=0.3,stratify=df['Sentiment'])

# val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, random_state = 2021, test_size = 0.5, stratify = temp_labels)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [6]:
from transformers import AutoModel,BertTokenizerFast

In [7]:
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# train_lens = [len(str(i).split()) for i in train_text]
# # plt.hist(train_lens)
# type(train_lens)

list

In [9]:
# pad_len = np.mean(train_lens)

In [26]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': tch.tensor(label, dtype=tch.long)
        }

In [ ]:
!pip install torch

In [38]:
import torch as tch
from torch import nn
from transformers import AdamW,BertForSequenceClassification
from torch.utils.data import DataLoader,Dataset

In [27]:
batch_size = 64
max_len = 128
epochs = 2
lr = 2e-5

train_dataset = CustomDataset(train_df['SentimentText'].values, train_df['Sentiment'].values, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_df['SentimentText'].values, test_df['Sentiment'].values, tokenizer, max_len)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [28]:
for pm in bert.parameters():
  pm.requires_grad = False


In [39]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
my_optimizer = AdamW(model.parameters(),lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = tch.device("cuda" if tch.cuda.is_available() else "cpu")

model.to(device)

my_train_dataset = CustomDataset(train_df['SentimentText'].values, train_df['Sentiment'].values, tokenizer, max_len)

train_dataloader = DataLoader(my_train_dataset,batch_size=64,shuffle=True)

# def train():
#   model.train()
#   total_loss,total_accuracy = 0,0
#   total_preds = []

#   for step,batch in enumerate(train_dataloader):
#     if step%50 == 0 and not step == 0:
#       print('Batch {:>5,} of {:>5,}.'.format(step,len(train_dataloader)))
#     batch = [r.to(device) for r in batch]
#     sent_id,mask,labels = batch
#     model.zero_grad()
#     preds = model(sent_id,mask)
#     loss = nn.CrossEntropyLoss(preds,labels)
#     total_loss += loss.item()
#     loss.backward()
#     tch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#     my_optimizer.step()
#     preds=preds.detach().cpu().numpy()
#     total_preds.append(preds)

#   avg_loss = total_loss / len(train_dataloader)

#   total_preds = np.concatenate(total_preds,axis=0)

#   return avg_loss,total_preds

from tqdm import tqdm

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}', leave=False)

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        my_optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        my_optimizer.step()

        progress_bar.set_postfix({'Batch Loss': loss.item()})

    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}')




Epoch 1:   0%|          | 0/19729 [00:00<?, ?it/s]

In [32]:
##evaluate model on validation data
my_test_dataset = CustomDataset(test_df['SentimentText'].values, test_df['Sentiment'].values, tokenizer, max_len)

test_dataloader = DataLoader(my_test_dataset,batch_size=64,shuffle=True)

test_predictions, test_labels = [], []
with tch.no_grad():
    for batch in tqdm(test_dataloader, desc='Test'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        _, predicted = tch.max(logits, 1)

        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())



In [34]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_labels, test_predictions)
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 1/5


AttributeError: 'str' object has no attribute 'to'